In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
K.set_session(K.tf.Session(config=K.tf.ConfigProto(intra_op_parallelism_threads=32, inter_op_parallelism_threads=32)))

from keras.applications import Xception
from keras import models
from keras import layers
from keras import optimizers

import numpy as np
import gc

"""
./format_data/train/
./format_data/val/
"""
train_dir = "./format_data/train/"
test_dir = "./format_data/val/"

feature_size = 299
batch_size = 8

Using TensorFlow backend.


In [2]:
train_gen = ImageDataGenerator(rescale=1./255,rotation_range = 360,fill_mode='reflect',
                               zoom_range=0.02,
                               width_shift_range = 0.2,height_shift_range = 0.2,
                               shear_range = 0.2,
                               horizontal_flip=True,vertical_flip=True)
test_gen = ImageDataGenerator(rescale=1./255,horizontal_flip=True,vertical_flip=True)

train_gener = train_gen.flow_from_directory(train_dir,target_size=(feature_size,feature_size),batch_size=batch_size,shuffle = True,seed = 100,class_mode="categorical")
test_gener = test_gen.flow_from_directory(test_dir,target_size=(feature_size,feature_size),batch_size=batch_size,class_mode="categorical")

for data_batch,label_batch in train_gener:
    print(data_batch.shape)
    print(label_batch.shape)
    break

Found 1886 images belonging to 11 classes.
Found 110 images belonging to 11 classes.
(8, 299, 299, 3)
(8, 11)


In [3]:
X_model = Xception(weights=None,include_top=False,input_shape=(feature_size,feature_size,3))
X_model.load_weights("./xception_weights_notop.h5")
X_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 299, 299, 3)   0                                            
____________________________________________________________________________________________________
block1_conv1 (Conv2D)            (None, 149, 149, 32)  864         input_1[0][0]                    
____________________________________________________________________________________________________
block1_conv1_bn (BatchNormalizat (None, 149, 149, 32)  128         block1_conv1[0][0]               
____________________________________________________________________________________________________
block1_conv1_act (Activation)    (None, 149, 149, 32)  0           block1_conv1_bn[0][0]            
___________________________________________________________________________________________

In [4]:
model = models.Sequential()

model.add(X_model)
model.add(layers.Flatten())
model.add(layers.Dense(512,activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(11,activation='softmax'))

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [5]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Model)             (None, 10, 10, 2048)      20861480  
_________________________________________________________________
flatten_1 (Flatten)          (None, 204800)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               104858112 
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 11)                5643      
Total params: 125,725,235
Trainable params: 125,670,707
Non-trainable params: 54,528
_________________________________________________________________


In [6]:
#below 10 epoch no need for warrying over fitting .. val_acc around 0.645
#best around 11
model.compile(loss='categorical_crossentropy',optimizer=optimizers.RMSprop(lr=1e-4),metrics=['acc'])
history = model.fit_generator(train_gener,steps_per_epoch=100,epochs=11,validation_data=test_gener,validation_steps=20)

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Epoch 1/11
100/100 [==============================] - 96s - loss: 2.0496 - acc: 0.5875 - val_loss: 4.5510 - val_acc: 0.1582
Epoch 2/11
100/100 [==============================] - 84s - loss: 1.0051 - acc: 0.7175 - val_loss: 3.0942 - val_acc: 0.2405
Epoch 3/11
100/100 [==============================] - 83s - loss: 0.7927 - acc: 0.7571 - val_loss: 4.9404 - val_acc: 0.3101
Epoch 4/11
100/100 [==============================] - 83s - loss: 0.6795 - acc: 0.8075 - val_loss: 2.4420 - val_acc: 0.4937
Epoch 5/11
100/100 [==============================] - 83s - loss: 0.4894 - acc: 0.8662 - val_loss: 2.7276 - val_acc: 0.5506
Epoch 6/11
100/100 [==============================] - 83s - loss: 0.4822 - acc: 0.8713 - val_loss: 2.6170 - val_acc: 0.5823
Epoch 7/11
100/100 [==============================] - 83s - loss: 0.4454 - acc: 0.8825 - val_loss: 1.3533 - va

In [7]:
model.save('Xception_whole.h5')